In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix
import re
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from sklearn.svm import SVC

# Mount Drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv("drive/MyDrive/ICHCL_baseline/hasoc2021-train/Hasoc2021_train.csv")
data.head()

,Unnamed: 0,_id,text,task_1,task_2
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,PRFN
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,NONE
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN


In [ ]:
tweet=data.text
t1=data.task_1
t2=data.task_2
tweet

0       @wealth if you made it through this &amp;&amp;...
1       Technically that's still turning back the cloc...
2       @VMBJP @BJP4Bengal @BJP4India @narendramodi @J...
3       @krtoprak_yigit Soldier of Japan Who has dick ...
4       @blueheartedly You'd be better off asking who ...
                              ...                        
3838    @BBCNews Let the dog deal with the wanker once...
3839    India has suffered a lot. That Chinese bastard...
3840    People didn't give 300+ seats majority to BJP ...
3841    @KanganaTeam This is such a vile, xenophobic a...
3842    @30iPpgStmILw0SI @ChinaDaily #ChineseVirus #Wu...
Name: text, Length: 3843, dtype: object

##PREPROCESSING AS STATED IN PAPER
####1.LOWER CASE CONVERTING
####2.REMOVING BAD SYMBOLS
####3.STOP WORD REMOVER
####4.PUNCTUATION REMOVER
####5.TOKENIZATION
####6.STEMMING

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])


In [ ]:
def preprocess(tweets):
  tweets=tweets.lower()                   #convert to lower case
  tweets = re.sub(r"http\S+", " ", tweets) #remove urls
  l=tweets.split(" ")
  for i in l:
    if '@' in i:
      l.remove(i)
  
  tweets=" ".join(l)
  tweets= re.sub('[^A-Za-z]+',' ',tweets) #remove bad character
  tweets = [word for word in tweets.split(" ") if not word in stop_words] # removing stop words
  tweets= [lemmatizer.lemmatize(token, "v") for token in tweets] #Lemmatization
  tweets=" ".join(tweets) 
  return tweets 
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text


In [ ]:
data["text"]=data["text"].apply(lambda x:decontract(x))
data["text"]=data["text"].apply(lambda x:preprocess(x))
data.text

0       make amp amp not able start make money sustain...
1            technically still turn back clock dick head 
2        bjp bengal narendramodi amitshah rahulsinhabj...
3                                 soldier japan dick head
4             would better ask think sleazy shitbag lmao 
                              ...                        
3838                          let dog deal wanker un arm 
3839    india suffer lot chinese bastard pay price chi...
3840    people not give seat majority bjp see bengalbu...
3841    vile xenophobic uneducated comment struggle be...
3842     chinadaily chinesevirus wuhanvirus correct na...
Name: text, Length: 3843, dtype: object

In [ ]:
def Nom2Num(tweet):
    if tweet == "HOF":
      return 1
    return 0

In [ ]:
data["task_1"] = data["task_1"].apply(lambda x: Nom2Num(x))


In [ ]:
data.iloc[3842]["text"]

' chinadaily chinesevirus wuhanvirus correct name pandemic shameless'

In [ ]:
data["task_1"]

0       1
1       1
2       0
3       1
4       1
       ..
3838    1
3839    1
3840    1
3841    1
3842    0
Name: task_1, Length: 3843, dtype: int64

USING TFIDF VECTORIZATION AND IDENTIFYING BI GRAMS

In [ ]:
vectorizer = CountVectorizer(ngram_range = (1,1)) #GETTING BIGRAMS
X1 = vectorizer.fit_transform(tweet) 
features = (vectorizer.get_feature_names())

In [ ]:
vectorizer = TfidfVectorizer(ngram_range = (1,1))
X2 = vectorizer.fit_transform(tweet)
scores = (X2.toarray())
scores

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
sums = X2.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0,col] ))
ranking = pd.DataFrame(data1, columns = ['term','rank'])
words = (ranking.sort_values('rank', ascending = False))

In [ ]:
words.head(10)

,term,rank
6000,not,105.773175
3286,fuck,83.702592
4171,india,75.550193
5555,modi,68.676249
4176,indiacovidcrisis,65.462046
6414,people,63.444964
7201,resignmodi,60.419056
3413,get,60.234595
911,bitch,57.267951
6582,pm,54.317766


In [ ]:
features

['aaa',
 'aaah',
 'aadhaar',
 'aadmi',
 'aaj',
 'aajtak',
 'aajtakteam',
 'aala',
 'aam',
 'aamaadmiparty',
 'aap',
 'aapi',
 'aapko',
 'aaps',
 'aapse',
 'aartic',
 'aata',
 'aatmanirbharbharat',
 'ab',
 'abacha',
 'abandon',
 'abang',
 'abate',
 'abc',
 'abd',
 'abdicate',
 'abdominal',
 'abdool',
 'abdul',
 'aber',
 'abhineta',
 'abhorrent',
 'ability',
 'abit',
 'able',
 'abolish',
 'abortion',
 'abound',
 'abpnews',
 'abroad',
 'abrogate',
 'abscond',
 'absence',
 'absent',
 'abso',
 'absolute',
 'absolutely',
 'absolutesystemfailure',
 'absurd',
 'abt',
 'abundance',
 'abuse',
 'abvnews',
 'aby',
 'ac',
 'academics',
 'accelerate',
 'accent',
 'accept',
 'acceptable',
 'access',
 'accident',
 'accidentally',
 'accolades',
 'accomplice',
 'accord',
 'accordingly',
 'account',
 'accountability',
 'accountable',
 'accurate',
 'accusation',
 'accusations',
 'accuse',
 'ache',
 'achieve',
 'acid',
 'acknowledge',
 'acknowledgingvtge',
 'acne',
 'acosta',
 'acquire',
 'across',
 'act',

In [ ]:
data.drop(["Unnamed: 0","_id","task_2"],inplace=True,axis=1)

In [ ]:
t1=data["task_1"]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X2,t1, test_size=0.3, random_state=42)
print("Training data size : ", X_train.shape)
print("Test data size : ", X_val.shape)

Training data size :  (2690, 9590)
Test data size :  (1153, 9590)


In [ ]:
data["task_1"].value_counts()

1    2501
0    1342
Name: task_1, dtype: int64

In [ ]:
clf = SVC(kernel='rbf')
clf.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred=clf.predict(X_val)

In [ ]:
print(accuracy_score(y_val,y_pred))

0.7788378143972247


In [ ]:
print('F1-score: ', f1_score(y_pred, y_val))

F1-score:  0.8455481526347668


In [ ]:
print('Confusion matrix:')
confusion_matrix(y_val,y_pred)

Confusion matrix:


array([[200, 182],
       [ 73, 698]])